In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet


In [51]:
df = pd.read_csv('tennis_atp-master/atp_matches_2023.csv')

In [52]:
df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2023-9900,United Cup,Hard,18,A,20230102,300,126203,3.0,NaN,...,62.0,47.0,15.0,12.0,9.0,9.0,9.0,3355.0,16.0,2375.0
1,2023-9900,United Cup,Hard,18,A,20230102,299,126207,NaN,NaN,...,12.0,8.0,3.0,4.0,1.0,3.0,19.0,2000.0,23.0,1865.0
2,2023-9900,United Cup,Hard,18,A,20230102,296,126203,3.0,NaN,...,62.0,51.0,7.0,12.0,2.0,2.0,9.0,3355.0,10.0,2905.0
3,2023-9900,United Cup,Hard,18,A,20230102,295,126207,NaN,NaN,...,41.0,26.0,12.0,9.0,6.0,9.0,19.0,2000.0,245.0,220.0
4,2023-9900,United Cup,Hard,18,A,20230102,292,126774,1.0,NaN,...,58.0,48.0,18.0,16.0,1.0,2.0,4.0,5550.0,16.0,2375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,2023-M-DC-2023-WG2-PO-RSA-LUX-01,Davis Cup WG2 PO: RSA vs LUX,NaN,4,D,20230204,5,202335,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1717.0,1.0
2982,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,1,117365,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,990.0,11.0,279.0,190.0
2983,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,2,121411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,364.0,131.0,894.0,15.0
2984,2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NaN,4,D,20230203,4,144949,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,894.0,15.0,285.0,184.0


In [53]:
pbp = pd.read_csv('tennis_slam_pointbypoint-master/2023-wimbledon-points.csv')

In [54]:
pbp['match_id'].unique()

array(['2023-wimbledon-1101', '2023-wimbledon-1102',
       '2023-wimbledon-1103', '2023-wimbledon-1104',
       '2023-wimbledon-1105', '2023-wimbledon-1106',
       '2023-wimbledon-1107', '2023-wimbledon-1108',
       '2023-wimbledon-1109', '2023-wimbledon-1110',
       '2023-wimbledon-1111', '2023-wimbledon-1112',
       '2023-wimbledon-1113', '2023-wimbledon-1114',
       '2023-wimbledon-1115', '2023-wimbledon-1116',
       '2023-wimbledon-1117', '2023-wimbledon-1118',
       '2023-wimbledon-1119', '2023-wimbledon-1120',
       '2023-wimbledon-1121', '2023-wimbledon-1122',
       '2023-wimbledon-1123', '2023-wimbledon-1124',
       '2023-wimbledon-1125', '2023-wimbledon-1126',
       '2023-wimbledon-1127', '2023-wimbledon-1128',
       '2023-wimbledon-1129', '2023-wimbledon-1130',
       '2023-wimbledon-1131', '2023-wimbledon-1132',
       '2023-wimbledon-1133', '2023-wimbledon-1134',
       '2023-wimbledon-1135', '2023-wimbledon-1136',
       '2023-wimbledon-1137', '2023-wimbledon-

In [55]:
charting = pd.read_csv('tennis_MatchChartingProject-master/charting-m-points-2020s.csv', encoding= 'latin1', low_memory=False)

In [56]:
charting    

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0,0,0-0,1 (1),1,0,...,NaN,False,False,False,True,False,False,1,1,1
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0,0,15-0,1 (2),1,0,...,1.0,False,False,True,False,False,False,2,0,8
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0,0,15-15,1 (3),1,0,...,1.0,False,False,True,False,False,False,2,0,4
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0,0,15-30,1 (4),1,0,...,NaN,False,False,True,False,False,False,1,1,3
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0,0,30-30,1 (5),1,0,...,NaN,False,False,True,False,False,False,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246625,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,203,1,1,5,2,15-0,31 (2),1,0,...,1.0,False,False,False,False,True,False,1,1,11
246626,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,204,1,1,5,2,30-0,31 (3),1,0,...,1.0,False,False,True,False,False,False,1,1,9
246627,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,205,1,1,5,2,40-0,31 (4),1,0,...,NaN,False,False,False,False,True,False,2,0,10
246628,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,206,1,1,5,2,40-15,31 (5),1,0,...,NaN,False,False,False,False,True,False,2,0,6


In [57]:
charting['match_id'].unique()

array(['20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton',
       '20230905-M-US_Open-QF-Novak_Djokovic_-Taylor_Fritz',
       '20230831-M-US_Open-R64-Hubert_Hurkacz-Jack_Draper', ...,
       '20200103-M-ATP_Cup-RR-Steve_Darcis-Alexander_Cozbinov',
       '20200103-M-ATP_Cup-RR-Stefanos_Tsitsipas-Denis_Shapovalov',
       '20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander_Zverev'],
      dtype=object)

In [58]:
charting['rallyCount'] = pd.to_numeric(charting['rallyCount'], errors='coerce')
charting['rallyCount'].mean()

4.145064043563409

In [59]:
#fix: charting.drop(charting.iloc[[187254]])
charting = charting.drop(charting.index[187254])

ValueError: Input data must be a pandas object to reorder